In [1]:
import numpy as np
import gzip
import itertools
from pathlib import Path
import os, sys
import re
import matplotlib.pyplot as plt
import arviz as az
%matplotlib inline 

In [2]:
#want to load up the chain of parameter values.
#this is stored in the P1-GMO-point.mc.gz file, 
#which holds an array of (n x m x [p+1]) values,
#where p is the number of parameters,
#n is the number of samples explored per walker,
#and m is the number of walkers.
#the first parameter stored is the -ve log likelihood of the parameters.

Q = [] 
with gzip.open(os.path.join(os.getcwd(), 'P1-GMO-point.mc.gz'), 'rt') as f: #defines the list 'files' as f and opens each file in the list and then closes it automatically#This block initialises a load of lists and variables that are used in the main for loop.
    file_content = f.read()
    splitted = file_content.splitlines()
    print(splitted[0])
    for line in itertools.islice(splitted, 1, None):
        list2 = line.strip(' ').strip('\n').split(' ')
        list3 = [float(i) for i in list2]
        Q.append((list3))

# logp point (Nthin x Npop x Nvar = [6001,200,20])


In [3]:
from uravu.distribution import Distribution

In [4]:
Qarray = np.array(Q)
print(Qarray.shape) #now we have all the samples per parameter in an array.

(1200200, 21)


In [5]:
#create a list of names for each parameter.

names = [
        'logl',
        'P1_GMO_ddod_25_Bkg',
        'P1_GMO_ddod_25_I0',
        'h_dod_1 interface',
        'd_dod_2 rho',
        'GMO_h_25 interface',
        'GMO_solv_25',
        'GMO_h_25 thickness',
        'FeOx_rough',
        'FeOx_thick',
        'Fe_rough',
        'Fe_thick',
        'SiOx thickness',
        'P1_GMO_CMdod_25 Bkg',
        'P1_GMO_CMdod_25 I0',
        'P1_GMO_hdod_25 Bkg',
        'P1_GMO_hdod_25 I0',
        'FeOx rho',
        'FeOx_m',
        'Fe rho',
        'Fe_magmom'
        ]

In [6]:
variables = []
for i in range (1, Qarray.shape[1]):
    variables.append(Distribution(Qarray[:, i], name=names[i], ci_points=(2.5, 97.5))) #16, 84 #2.5, 97.5

In [11]:
#calculate the sld of the solvent in the CMdod contrast.

CMdod = Distribution(Qarray[:, [4]]*0.352 +(1-0.352)*-0.46, name='CMdod', ci_points=(2.5, 97.5))

print(f'{CMdod.n:.6E}')
print(CMdod.n-CMdod.con_int[0])
print(CMdod.con_int[1]-CMdod.n)

1.926268E+00
0.0078059692083172205
0.014917672802755355


In [12]:
#print out the statistics that describe each parameter.

import scipy

alpha = 0.001
for i, j in enumerate(variables):
    print(f"""{variables[i].name}
    Median: {j.n:.3E}
    Lower: {j.con_int[0]:.3E}
    Upper: {j.con_int[1]:.3E}
    Minus: {j.n-j.con_int[0]:.3E}
    Plus: {j.con_int[1]-j.n:.3E}
    Is gauss?: {'Yes' if scipy.stats.normaltest(j.samples)[1] >= alpha else 'No'}
    Gauss_test_p: {scipy.stats.normaltest(j.samples)[1]:.3E}
    MCSE_median: {az.mcse(j.samples, method="quantile", prob=0.5):.3E}
    MCSE_lb: {az.mcse(j.samples, method="quantile", prob=0.025):.3E}
    MCSE_ub: {az.mcse(j.samples, method="quantile", prob=0.975):.3E}""")

P1_GMO_ddod_25_Bkg
    Median: 4.850E-06
    Lower: 3.982E-06
    Upper: 5.678E-06
    Minus: 8.684E-07
    Plus: 8.281E-07
    Is gauss?: No
    Gauss_test_p: 1.219E-200
    MCSE_median: 4.974E-10
    MCSE_lb: 1.077E-09
    MCSE_ub: 9.757E-10
P1_GMO_ddod_25_I0
    Median: 9.483E-01
    Lower: 9.414E-01
    Upper: 9.552E-01
    Minus: 6.963E-03
    Plus: 6.866E-03
    Is gauss?: No
    Gauss_test_p: 4.561E-24
    MCSE_median: 4.041E-06
    MCSE_lb: 8.066E-06
    MCSE_ub: 7.998E-06
h_dod_1 interface
    Median: 5.978E+00
    Lower: 3.863E+00
    Upper: 7.526E+00
    Minus: 2.114E+00
    Plus: 1.548E+00
    Is gauss?: No
    Gauss_test_p: 0.000E+00
    MCSE_median: 1.150E-03
    MCSE_lb: 2.720E-03
    MCSE_ub: 1.147E-03
d_dod_2 rho
    Median: 6.319E+00
    Lower: 6.297E+00
    Upper: 6.362E+00
    Minus: 2.218E-02
    Plus: 4.238E-02
    Is gauss?: No
    Gauss_test_p: 0.000E+00
    MCSE_median: 2.335E-05
    MCSE_lb: 6.874E-06
    MCSE_ub: 5.027E-05
GMO_h_25 interface
    Median: 4.796

In [10]:
# lets calculate the surface excess of GMO and the area per molecule

GMOscattlen = 0.00013218 #Å
Avogad = 6.0221409*1E23
GMO_SE = Distribution((((Qarray[:, [7]]*((100-Qarray[:, [6]])/100)*0.21*1E-6)/(GMOscattlen*Avogad))*1E20), name='GMO_SE', ci_points=(2.5, 97.5))

print(f"""{GMO_SE.name}
      Median: {GMO_SE.n:.2E}
      Lower: {GMO_SE.con_int[0]:.2E}
      Upper: {GMO_SE.con_int[1]:.2E}
      Minus: {GMO_SE.n-GMO_SE.con_int[0]:.2E}
      Plus: {GMO_SE.con_int[1]-GMO_SE.n:.2E}""")

GMO_APM = Distribution(1E20*(1/GMO_SE.samples)/Avogad, name='GMO_APM', ci_points=(2.5, 97.5))
print(f"""{GMO_APM.name}
      Median: {GMO_APM.n:.2E}
      Lower: {GMO_APM.con_int[0]:.2E}
      Upper: {GMO_APM.con_int[1]:.2E}
      Minus: {GMO_APM.n-GMO_APM.con_int[0]:.2E}
      Plus: {GMO_APM.con_int[1]-GMO_APM.n:.2E}""")

GMO_SE
      Median: 4.68E-06
      Lower: 4.55E-06
      Upper: 4.81E-06
      Minus: 1.27E-07
      Plus: 1.31E-07
GMO_APM
      Median: 3.55E+01
      Lower: 3.45E+01
      Upper: 3.65E+01
      Minus: 9.65E-01
      Plus: 9.91E-01


In [11]:
%matplotlib qt

import corner

#get the label for each parameter.

plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.rc('font', serif='Computer Modern Roman')
plt.rc('font', size=20)          # controls default text sizes

#create fancy labels for corner plot
labels = [
        r'GMO $\sigma{}$ / $\mathrm{\AA{}}$',
        r'dod-d$_{26}$ $\beta_{\mathrm{n}}$ /' '\n' r'$\mathrm{\AA{}}^{-2}\times{}10^{-6}$',
        r'FeO$_{x}$ $\sigma{}$ / $\mathrm{\AA{}}$',
        r'GMO $\phi{}$ / \%',
        r'GMO $d$ / $\mathrm{\AA{}}$',
        r'Fe $\sigma{}$ / $\mathrm{\AA{}}$',
        r'FeO$_{x}$ $d$ / $\mathrm{\AA{}}$',
        r'SiO$_{2}$ $\sigma{}$ / $\mathrm{\AA{}}$',
        r'Fe $d$ / $\mathrm{\AA{}}$',
        r'SiO$_{2}$ $d$ / $\mathrm{\AA{}}$',
        r'FeO$_{x}$ $\beta_{\mathrm{n}}$ /' '\n' r'$\mathrm{\AA{}}^{-2}\times{}10^{-6}$',
        r'FeO$_{x}$ $\beta_{\mathrm{m}}$ /' '\n' r'$\mathrm{\AA{}}^{-2}\times{}10^{-6}$',
        r'Fe $\beta_{\mathrm{n}}$ /' '\n' r'$\mathrm{\AA{}}^{-2}\times{}10^{-6}$',
        r'Fe $\mu$ / $\mu_{\mathrm{B}}$'
        ]

#get the chain of sampels for each parameter and reshape it for the corner plot.
chain_corner = Qarray #this ends up with a matrix that is
# n x m - n = number of samples per parameter and m is the number of parameters.

#make some grayscale colors for the contour map we'll be looking at...
color_cont = {'colors':['k', 'k', 'k', 'k'], 'linewidths': 0.87}
color_cont_fill = {'colors':['w', '#00000033', '#00000059', '#00000080', '#000000a6']}

title_font = {'fontsize': 20}

#don't include the intensity or background parameters in the corner plot.
chain_corner_focused = np.concatenate((chain_corner[:, 3:13], chain_corner[:, 17:22]), axis=1)

figure = corner.corner(chain_corner_focused, titles=labels, show_titles=True, quantiles=[0.025, 0.5, 0.975], 
                      plot_contours=True, color='k', plot_datapoints=True, plot_density=False, title_fmt=None,
                      max_n_ticks=2, fill_contours=True, contour_kwargs=color_cont, contourf_kwargs=color_cont_fill,
                      title_kwargs={'pad':'10', 'fontdict':title_font})